In [8]:
#!pip install transformers accelerate bitsandbytes
!pip install --upgrade transformers accelerate bitsandbytes

In [3]:
!pip install scipy

In [4]:
#!pip install sentencepiece
!pip show sentencepiece
!pip install --upgrade sentencepiece



Name: sentencepiece
Version: 0.2.0
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /home/codemaster29/miniconda3/lib/python3.12/site-packages
Requires: 
Required-by: 


In [5]:
!pip install einops torchvision

In [6]:
!pip install xformers

In [7]:
#!sudo pacman -S gcc -y


In [7]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 4070 SUPER


In [1]:
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# Load the tokenizer for the 'vicuna-7b-v1.5' model
tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

# Initialize an empty model to prepare for loading
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        'THUDM/cogvlm-chat-hf',
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        quantization_config=quantization_config,
    )

# Load the model weights and dispatch across available devices
model = load_checkpoint_and_dispatch(
    model,
    'THUDM/cogvlm-chat-hf',
    device_map="auto",
    no_split_module_classes=["CogVLMDecoderLayer"],  # Adjust this if needed
    dtype=torch.float16,
    offload_folder="offload",
)

# Enable gradient checkpointing if available
if hasattr(model, "enable_gradient_checkpointing"):
    model.enable_gradient_checkpointing()

# The model is now loaded with optimizations for lower memory usage
model.eval()  # Set to evaluation mode

print(model.hf_device_map)  # This will show you the device mapping

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 11.62 GiB of which 77.38 MiB is free. Including non-PyTorch memory, this process has 10.87 GiB memory in use. Of the allocated memory 10.62 GiB is allocated by PyTorch, and 48.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [1]:
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer

# Load the tokenizer for the 'vicuna-7b-v1.5' model
tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')

# Load the quantized model directly
model = AutoModelForCausalLM.from_pretrained(
    "Rodeszones/CogVLM-grounding-generalist-hf-quant4",
    trust_remote_code=True,
    device_map="auto",  # This will automatically distribute the model across available devices
    torch_dtype=torch.float16,  # Use float16 for additional memory savings
    load_in_8bit=True, # Load the model in 8-bit 
    offload_folder="offload", # Specify the folder to offload to
    load_in_8bit_fp32_cpu_offload=True # Enable CPU offloading for 32-bit parts
)

# Enable gradient checkpointing if available (optional, for fine-tuning)
if hasattr(model, "enable_gradient_checkpointing"):
    model.enable_gradient_checkpointing()

# Set the model to evaluation mode
model.eval()

# Print the device map to see how the model is distributed
print(model.hf_device_map)

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

configuration_cogvlm.py:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Rodeszones/CogVLM-grounding-generalist-hf-quant4:
- configuration_cogvlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_cogvlm.py:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

visual.py:   0%|          | 0.00/5.25k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Rodeszones/CogVLM-grounding-generalist-hf-quant4:
- visual.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Rodeszones/CogVLM-grounding-generalist-hf-quant4:
- modeling_cogvlm.py
- visual.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/codemaster29/miniconda3/lib/pyt

model.safetensors.index.json:   0%|          | 0.00/440k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.50G [00:00<?, ?B/s]

TypeError: CogVLMForCausalLM.__init__() got an unexpected keyword argument 'load_in_8bit_fp32_cpu_offload'

In [6]:
import requests
from PIL import Image
from io import BytesIO

def generate(query, img_url):
    # Fetch the image from the URL
    response = requests.get(img_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content)).convert('RGB')
        
        # Continue with the model processing...
        inputs = model.build_conversation_input_ids(
            tokenizer, query=query, history=[], images=[image], template_version='vqa'
        )
        inputs = {
            'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
            'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
            'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
            'images': [[inputs['images'][0].to('cuda').to(torch.bfloat16)]],
        }
    else:
        print("Failed to fetch image from URL.")


In [ ]:
query = 'Describe this image in detail'
image = 'https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg'
generate(query, image)